# Vibration Environment

We can use the IMU to determine what the vibration specrum was like during the flight. Looking at all axises of the accelerometer in the IMU we see the plots get significantly noisy during the boost portion of the flight.

In [ ]:
from numpy import subtract, divide, multiply, median, std, var, sqrt
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('png', 'pdf')
import launch

accel = []
for i, t in enumerate(launch.adis.time):
    accel.append(sqrt((launch.adis.acc_x[i]*launch.adis.acc_x[i]) + (launch.adis.acc_y[i]*launch.adis.acc_y[i]) + (launch.adis.acc_z[i]*launch.adis.acc_z[i])))

fig, ax1 = plt.subplots(figsize=(16,6))
plt.title(r"IMU Acceleration Magnitude")
plt.ylabel(r"Measured Acceleration [m/s$^2$]")
plt.xlabel(r"Mission Elapsed Time [s]")

plt.plot(launch.adis.time, launch.adis.acc_x, alpha=0.75, label="X (Up)")
plt.plot(launch.adis.time, launch.adis.acc_y, alpha=0.6, label="Y")
plt.plot(launch.adis.time, launch.adis.acc_z, alpha=0.6, label="Z")

plt.xlim([-5,42])
plt.legend()
plt.show()

In [ ]:
fig, ax1 = plt.subplots(figsize=(16,6))
plt.title(r"IMU Acceleration Magnitude")
plt.ylabel(r"Measured Acceleration [m/s$^2$]")
plt.xlabel(r"Mission Elapsed Time [s]")

plt.plot(launch.adis.time, accel, alpha=0.75, label="Up")

plt.xlim([-5,42])
#plt.legend()
plt.show()

If we take a Fourier transform of the acceleration magnitude data then we can see the spectral power over many frequencies during the flight.

In [ ]:
from scipy import signal

f, Pxx_den = signal.periodogram(divide(accel, launch.g_0), launch.adis.fs,  scaling='spectrum')

fig, ax1 = plt.subplots(figsize=(16,6))
plt.title(r"IMU Acceleration Power Spectral Density")
plt.ylabel(r"PSD [g${}^2$/Hz]")
plt.xlabel(r"Frequency [Hz]")

plt.semilogy(f[1:], Pxx_den[1:], alpha=0.75, label="Up")

plt.xlim([-1,launch.adis.fs/2])
plt.ylim([10e-14,1])
#plt.legend()
plt.show()

## Waterfall Spectrograph

The most interesting thing is to look at the change in vibration power spectrum over time. We use a waterfall chart to do this with sliding windows:

In [ ]:
from matplotlib.colors import LogNorm
from scipy.interpolate import interp1d
from matplotlib import cm
import numpy as np
from matplotlib import gridspec

data = divide(accel, launch.g_0)
fft_size = 1024
overlap_fac = 0.75

hop_size = np.int32(np.floor(fft_size * (1-overlap_fac)))
pad_end_size = fft_size          # the last segment can overlap the end of the data array by no more than one window size
total_segments = np.int32(np.ceil(len(data) / np.float32(hop_size)))
t_max = len(data) / np.float32(launch.adis.fs)

window = np.hanning(fft_size)  # our half cosine window
inner_pad = np.zeros(fft_size) # the zeros which will be used to double each segment size

proc = np.concatenate((data, np.zeros(pad_end_size)))      # the data to process
result = np.empty((total_segments, 513), dtype=np.float32) # space to hold the result

for i in xrange(total_segments):
    current_hop = hop_size * i                        # figure out the current segment offset
    segment = proc[current_hop:current_hop+fft_size]  # get the current segment
    windowed = segment * window                       # multiply by the half cosine function
    f, pxx    = signal.periodogram(windowed, launch.adis.fs)      # scipi spectrum function on windowed segment
    result[i, :] = pxx


# Start Figure
fig = plt.figure(figsize=(16,8))
plt.subplots_adjust(wspace=0.001)   # no space between horizontal charts
gs = gridspec.GridSpec(1, 2, width_ratios=[1, 7]) # stretch main chart to be most of the width

# Plot raw accel guide-chart
ax1 = plt.subplot(gs[0])
plt.title(r"")
plt.ylabel(r"Mission Elapsed Time [s]")
plt.xlabel(r"Acceleration [g]")
ax1.plot(data, launch.adis.time, 'k-', alpha=0.6, lw=0.4)
plt.xlim([-0.7,13.9])
plt.ylim([launch.adis.time[-1]+0.75,launch.adis.time[0]])

# Plot spetrogram
ax2 = plt.subplot(gs[1])
plt.title(r"IMU Acceleration Power Spectrum")
plt.xlabel(r"Vibration Frequency [Hz]")
img = ax2.imshow(result, origin='upper', cmap='plasma', interpolation='bicubic', aspect='auto', norm=LogNorm(vmax=1.0e-2, vmin=1.0e-10))

# Set sane ticks
tics = range(0,401,50)
ytics = range(0,41,10)
timebase = interp1d([launch.adis.time[0],launch.adis.time[-1]+1],[0,total_segments])
plt.xticks([t*(513/(launch.adis.fs/2.0)) for t in tics], tics)
plt.yticks([timebase(t) for t in ytics], ytics)

# Colors and grid
ax2.tick_params(axis='x', colors='white', labelcolor='black', length=5)
ax2.tick_params(axis='y', colors='white', labelcolor='black', length=5)
ax2.grid(color='#ffffff', alpha=0.8)
ticklabels = ax2.get_yticklabels()
plt.setp(ticklabels, visible=False)
plt.xlim([0,513])

# Colorbar tight placement
cbaxes = fig.add_axes([0.91, 0.125, 0.02, 0.775]) 
cbar = plt.colorbar(img, cax = cbaxes)
cbar.ax.set_ylabel(r'Vibration Power [g${}^2 /$ Hz]')

plt.show()